# Import Libraries

In [1]:
import pandas as pd
import numpy as np

# Loading the datasets

In [2]:
income_df = pd.read_csv('income_df_cleaned.csv')
medical_centers_df = pd.read_csv('medical_centers_df_cleaned.csv')
population_df = pd.read_csv('population_df_cleaned.csv')

# Merging the datasets on the 'neighborhood_name' and 'district_name' columns

I will discart the year column for the analysis. 

In [3]:
# Dropping the 'year' column from all dataframes, if it exists
income_df.drop(columns='year', errors='ignore', inplace=True)
medical_centers_df.drop(columns='year', errors='ignore', inplace=True)
population_df.drop(columns='year', errors='ignore', inplace=True)

#### Check if the values of the neighborhood_name column match

In [4]:
# Get unique values from the neighborhood_name column of each DataFrame
unique_neighborhood_income = set(income_df['neighborhood_name'].unique())
unique_neighborhood_medical_centers = set(medical_centers_df['neighborhood_name'].unique())
unique_neighborhood_population = set(population_df['neighborhood_name'].unique())

# Find the differences between the sets of unique values
differences_income_medical_centers = unique_neighborhood_income.symmetric_difference(unique_neighborhood_medical_centers)
differences_income_population = unique_neighborhood_income.symmetric_difference(unique_neighborhood_population)

# Print the found differences
if len(differences_income_medical_centers) > 0:
    print("Differences between 'neighborhood_name' in income and medical_centers:")
    print(differences_income_medical_centers)

if len(differences_income_population) > 0:
    print("Differences between 'neighborhood_name' in income and population:")
    print(differences_income_population)

Differences between 'neighborhood_name' in income and medical_centers:
{'Sants - Badal', 'Sant Gervasi - la Bonanova', 'Sant Gervasi- la Bonanova', 'Sants-Badal', 'Sant Gervasi- Galvany', 'Sant Gervasi - Galvany'}
Differences between 'neighborhood_name' in income and population:
{'Sants - Badal', 'Sant Gervasi - la Bonanova', 'Sant Gervasi- la Bonanova', 'Sants-Badal', 'Sant Gervasi- Galvany', 'Sant Gervasi - Galvany'}


#### Standardize the 'neighborhood_name' values across DataFrames

In [5]:
# Define a mapping of inconsistent names to standardized names
name_mapping = {
    'Sant Gervasi- la Bonanova': 'Sant Gervasi - la Bonanova',
    'Sants-Badal': 'Sants - Badal',
    'Sant Gervasi- Galvany': 'Sant Gervasi - Galvany',
}

# Apply the mapping to the 'neighborhood_name' column in each DataFrame
income_df['neighborhood_name'] = income_df['neighborhood_name'].replace(name_mapping)
medical_centers_df['neighborhood_name'] = medical_centers_df['neighborhood_name'].replace(name_mapping)
population_df['neighborhood_name'] = population_df['neighborhood_name'].replace(name_mapping)

In [6]:
# Check if the 'neighborhood_name' values match in all three DataFrames
matching_neighborhoods = set(income_df['neighborhood_name']).intersection(
    medical_centers_df['neighborhood_name'], population_df['neighborhood_name']
)

if len(matching_neighborhoods) == len(set(income_df['neighborhood_name'])):
    print("All 'neighborhood_name' values match in the three DataFrames.")
else:
    print("There are still differences in 'neighborhood_name' values.")

All 'neighborhood_name' values match in the three DataFrames.


#### Check if the values of the district_name column match

In [7]:
# Extract the 'district_name' column from each DataFrame and convert it to a set
income_districts = set(income_df['district_name'])
medical_centers_districts = set(medical_centers_df['district_name'])
population_districts = set(population_df['district_name'])

# Check if the 'district_name' sets are equal for all three DataFrames
if income_districts == medical_centers_districts == population_districts:
    print("The 'district_name' column matches between all three DataFrames.")
else:
    print("The 'district_name' column does not match between all three DataFrames.")

The 'district_name' column matches between all three DataFrames.


Now we can merge the DataFrames.

## Merge DataFrames

In [8]:
# Merge the DataFrames on 'neighborhood_name' and 'district_name' columns
merged_df = income_df.merge(medical_centers_df, on=['neighborhood_name', 'district_name'], how='inner')
merged_df = merged_df.merge(population_df, on=['neighborhood_name', 'district_name'], how='inner')

# Now, merged_df contains the merged data based on both 'neighborhood_name' and 'district_name'

In [9]:
merged_df

,neighborhood_name,district_name,median_income,medical_center,type,admin,population,residential_area_ha,net_density_hab_ha
0,Baró de Viver,Sant Andreu,29438.5,None,None,None,2645,3.9,673
1,Can Baró,Horta-Guinardó,45922.0,None,None,None,9233,14.3,645
2,Can Peguera,Nou Barris,28117.0,None,None,None,2204,6.1,359
3,Canyelles,Nou Barris,43158.0,None,None,None,6797,11.0,620
4,Ciutat Meridiana,Nou Barris,29393.5,None,None,None,11026,15.1,729
...,...,...,...,...,...,...,...,...,...
68,la Vila Olímpica del Poblenou,Sant Martí,83317.0,Centre d'Atenció Primària Vila Olímpica,CAP,Public,9240,24.3,380
69,la Vila de Gràcia,Gràcia,50733.0,"Clínica Sanza, Centre d'Atenció Primària Vila...","Other, CAP, Other","Private, Public, Private",49492,84.0,589
70,les Corts,Les Corts,65816.0,"Clínica Institut Marquès, Hospital de Barcelon...","Other, Hospital, Hospital, CAP","Private, Private, Private, Public",45422,64.7,702
71,les Roquetes,Nou Barris,31584.0,Centre d'Atenció Primària Roquetes,CAP,Public,16373,18.2,897


# Check on the main features

In [10]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 72
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   neighborhood_name    73 non-null     object 
 1   district_name        73 non-null     object 
 2   median_income        73 non-null     float64
 3   medical_center       73 non-null     object 
 4   type                 73 non-null     object 
 5   admin                73 non-null     object 
 6   population           73 non-null     int64  
 7   residential_area_ha  73 non-null     float64
 8   net_density_hab_ha   73 non-null     int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 5.7+ KB


We don't have null values and data types are correct.

In [11]:
# To check for duplicates in all the rows:
duplicates = merged_df[merged_df.duplicated()]
duplicates.count()

neighborhood_name      0
district_name          0
median_income          0
medical_center         0
type                   0
admin                  0
population             0
residential_area_ha    0
net_density_hab_ha     0
dtype: int64

We don't have duplicates.

In [12]:
# For loop to iterate through all columns 
# View unique values and count values
for column in merged_df.columns:
    unique_values = merged_df[column].unique()
    value_counts = merged_df[column].value_counts()
    print(f"Column: {column}")
    print("Unique Values:")
    print(unique_values)
    print("Value Counts:")
    print(value_counts)

Column: neighborhood_name
Unique Values:
['Baró de Viver' 'Can Baró' 'Can Peguera' 'Canyelles' 'Ciutat Meridiana'
 'Diagonal Mar i el Front Marítim del Poblenou' 'Horta' 'Hostafrancs'
 'Montbau' 'Navas' 'Pedralbes' 'Porta' 'Provençals del Poblenou'
 'Sant Andreu' 'Sant Antoni' 'Sant Genís dels Agudells'
 'Sant Gervasi - Galvany' 'Sant Gervasi - la Bonanova'
 'Sant Martí de Provençals' 'Sant Pere, Santa Caterina i la Ribera'
 'Sants' 'Sants - Badal' 'Sarrià' 'Torre Baró' 'Vallbona'
 'Vallcarca i els Penitents' 'Vallvidrera, el Tibidabo i les Planes'
 'Verdun' 'Vilapicina i la Torre Llobeta' 'el Baix Guinardó'
 'el Barri Gòtic' 'el Besòs i el Maresme' 'el Bon Pastor'
 "el Camp d'en Grassot i Gràcia Nova" "el Camp de l'Arpa del Clot"
 'el Carmel' 'el Clot' 'el Coll' 'el Congrés i els Indians'
 'el Fort Pienc' 'el Guinardó' 'el Parc i la Llacuna del Poblenou'
 'el Poble Sec' 'el Poblenou' 'el Putxet i el Farró' 'el Raval'
 'el Turó de la Peira' "l'Antiga Esquerra de l'Eixample" 'la Barcelo

In Barcelona, there are 10 districts, and when checking the 'district_name' column, we see that everything is correct because there are 10 unique values that correctly represent the names of the 10 districts.

Also in the 'neighborhood_name' column, we observe that the 73 neighborhoods of Barcelona are listed.

### I will change the median_income type from float to int.

In [13]:
merged_df['median_income'] = merged_df['median_income'].astype(int)

In [14]:
merged_df['median_income'].dtypes

dtype('int64')

# Split into Categorical and Numerical

In [15]:
# Select categorical columns (object and category data types)
categorical_df = merged_df.select_dtypes(include=['object', 'category'])

# Select numerical columns (int and float data types)
numerical_df = merged_df.select_dtypes(include=['int', 'float'])

In [16]:
numerical_df

,median_income,population,residential_area_ha,net_density_hab_ha
0,29438,2645,3.9,673
1,45922,9233,14.3,645
2,28117,2204,6.1,359
3,43158,6797,11.0,620
4,29393,11026,15.1,729
...,...,...,...,...
68,83317,9240,24.3,380
69,50733,49492,84.0,589
70,65816,45422,64.7,702
71,31584,16373,18.2,897


In [17]:
categorical_df

,neighborhood_name,district_name,medical_center,type,admin
0,Baró de Viver,Sant Andreu,None,None,None
1,Can Baró,Horta-Guinardó,None,None,None
2,Can Peguera,Nou Barris,None,None,None
3,Canyelles,Nou Barris,None,None,None
4,Ciutat Meridiana,Nou Barris,None,None,None
...,...,...,...,...,...
68,la Vila Olímpica del Poblenou,Sant Martí,Centre d'Atenció Primària Vila Olímpica,CAP,Public
69,la Vila de Gràcia,Gràcia,"Clínica Sanza, Centre d'Atenció Primària Vila...","Other, CAP, Other","Private, Public, Private"
70,les Corts,Les Corts,"Clínica Institut Marquès, Hospital de Barcelon...","Other, Hospital, Hospital, CAP","Private, Private, Private, Public"
71,les Roquetes,Nou Barris,Centre d'Atenció Primària Roquetes,CAP,Public


# Saving

In [18]:
# Reset the index of merged_df
merged_df = merged_df.reset_index(drop=True)

# Save the DataFrame to a CSV file
merged_df.to_csv('merged_data.csv', index=False)